## Random CuInZnS Nanocrystal Generator

Generates 34-cation NCs with randomly distributed Cu, In, and Zn cations.  
Constraints: Number of Cu == Number of In, and number of surface H on Cu == number of surface H on In.

In [1]:
from random import randint
import numpy as np
import os

In [2]:
In_cations = [1, 8, 11, 12, 16, 20, 31, 32, 36, 40, 45, 46, 48, 51, 56, 63, 66]
Cu_cations = [4, 5, 7, 17, 21, 24, 25, 30, 33, 37, 39, 47, 52, 55, 59, 60, 67]
cations = sorted(Cu_cations + In_cations)
corner_cations = [63, 66, 67]
surface_cations = [11, 21, 24, 25, 30, 31, 32, 33, 36,
           37, 39, 40, 45, 46, 47, 48, 51, 52, 55, 56, 59, 60]

In [3]:
with open('CuInZnS34_random_template.txt') as f:
    temp = f.readlines()
atom_coord_lines = temp[0:68]
H_coord_lines = temp[68:124]
SH_charge_lines = temp[125:153]

0 = S  
1 = Cu  
2 = Zn  
3 = In  

In [4]:
def generate_structure2(num_CuIn):
    
    bad_structure = True
    attempts = 0

    while bad_structure: 
        structure = np.zeros(68)
        
        for site in cations:
            structure[site -1] = 2
        
        for i in range(num_CuIn):
            Cu = Cu_cations[randint(0, len(Cu_cations)-1)] - 1  # pick a site for Cu
            In = In_cations[randint(0, len(In_cations)-1)] - 1  # pick a site for In
            
            while structure[Cu] == 1:  # if there is already Cu in the selected site
                Cu = Cu_cations[randint(0, len(Cu_cations)-1)] - 1  # pick a new site
            while structure[In] == 3:  # if there is already In in the selected site
                In = In_cations[randint(0, len(In_cations)-1)] - 1  # pick a new site
            
            structure[Cu] = 1
            structure[In] = 3
        
        unique, counts = np.unique(structure, return_counts=True)
        attempts += 1
        Cu_H = 0
        In_H = 0
        for i, atom in enumerate(structure):
            if atom == 1:  # copper
                if (i+1) in corner_cations:
                    Cu_H += 2
                elif (i+1) in surface_cations:
                    Cu_H += 1
            elif atom == 3:  # indium
                if (i+1) in corner_cations:
                    In_H += 2
                elif (i+1) in surface_cations:
                    In_H += 1


        if (counts[1] == counts[3]) and (Cu_H == In_H):
            print("Cu{} In{} Zn{}".format(counts[1], counts[3], counts[2]))
            print("Attempts: {}".format(attempts))
            return structure
        

In [5]:
def generate_structure(num_CuIn=0):

    bad_structure = True
    attempts = 0

    while bad_structure: 
        structure = np.zeros(68)
        for i in range(len(structure)):
            if (i+1) in Cu_cations:
                structure[i] = randint(1,2)
            if (i+1) in In_cations:
                structure[i] = randint(2,3)
        attempts += 1
        unique, counts = np.unique(structure, return_counts=True)
        
        if (num_CuIn==0 or num_CuIn==counts[1]): # only continue if there is no constraint on Cu, In
                                                 # or if the structure meets the constraint
            Cu_H = 0
            In_H = 0
            for i, atom in enumerate(structure):
                if atom == 1:  # copper
                    if (i+1) in corner_cations:
                        Cu_H += 2
                    elif (i+1) in surface_cations:
                        Cu_H += 1
                elif atom == 3:  # indium
                    if (i+1) in corner_cations:
                        In_H += 2
                    elif (i+1) in surface_cations:
                        In_H += 1


            if (counts[1] == counts[3]) and (Cu_H == In_H):
                print("Cu{} In{} Zn{}".format(counts[1], counts[3], counts[2]))
                print("Attempts: {}".format(attempts))
                return structure

In [6]:
def write_structure(structure):
    
    unique, counts = np.unique(structure, return_counts=True)
    filenum = 1
    filename = "ZnS{}_CuIn{}_rand{}_opt".format(counts[2], counts[1], filenum)
    print(filename)
    
    while os.path.exists(filename + ".gjf"):
        filenum += 1
        filename = "ZnS{}_CuIn{}_rand{}_opt".format(counts[2], counts[1], filenum)
    
    with open(filename + ".gjf", 'w') as f:

        f.write("%chk={}.chk\n".format(filename))
        f.write("%mem=60GB\n")
        f.write("%nprocshared=16\n")
        f.write("#p pbe1pbe/lanl2dz opt massage scf=conver=5\n\n")
        f.write("Random Zn{} Cu{} In{} structure {}\n\n".format(
                counts[2], counts[1], counts[3], filenum))
        f.write("0 1\n")

        for i, atom in enumerate(structure):
            if atom == 0:
                atom_type = "S"
            elif atom == 1:
                atom_type = "Cu"
            elif atom == 2:
                atom_type = "Zn"
            elif atom == 3:
                atom_type = "In"
            xyz = atom_coord_lines[i].split()
            f.write("{:2}\t{: f}\t{: f}\t{: f}\n".format(atom_type, float(xyz[1]), float(xyz[2]), float(xyz[3])))


        for line in H_coord_lines:
            xyz = line.split()
            f.write("{:2}\t{: f}\t{: f}\t{: f}\n".format(xyz[0], float(xyz[1]), float(xyz[2]), float(xyz[3])))

        f.write("\n")

        for line in SH_charge_lines:
            line = line.split()
            f.write("{}\t{}\t{}\n".format(line[0], line[1], line[2]))

        H_num = 97
        total_H_charge = 0
        for i in range(67,0,-1):
            atom = structure[i]
            if atom == 1:
                H_charge = 1.75
            elif atom == 2:
                H_charge = 1.5
            elif atom == 3:
                H_charge = 1.25

            if ((i+1) in corner_cations) or ((i+1) in surface_cations):
                f.write("{}\tznuc\t{}\n".format(H_num, H_charge))
                H_num += 1
                total_H_charge += H_charge
            if ((i+1) in corner_cations):  # do a second time if on corner
                f.write("{}\tznuc\t{}\n".format(H_num, H_charge))
                H_num += 1
                total_H_charge += H_charge

In [9]:
test = generate_structure(10)
write_structure(test)

Cu10 In10 Zn14
Attempts: 113
ZnS14_CuIn10_rand1_opt


In [12]:
test2 = generate_structure2(1)
write_structure(test2)

Cu1 In1 Zn32
Attempts: 2
ZnS32_CuIn1_rand1_opt


In [11]:
list(range(1,17))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

In [12]:
for i in range(1,17):
    write_structure(generate_structure2(i))

Cu1 In1 Zn32
Attempts: 2
ZnS32_CuIn1_rand1_opt
Cu2 In2 Zn30
Attempts: 2
ZnS30_CuIn2_rand1_opt
Cu3 In3 Zn28
Attempts: 2
ZnS28_CuIn3_rand1_opt
Cu4 In4 Zn26
Attempts: 1
ZnS26_CuIn4_rand1_opt
Cu5 In5 Zn24
Attempts: 3
ZnS24_CuIn5_rand1_opt
Cu6 In6 Zn22
Attempts: 1
ZnS22_CuIn6_rand1_opt
Cu7 In7 Zn20
Attempts: 1
ZnS20_CuIn7_rand1_opt
Cu8 In8 Zn18
Attempts: 3
ZnS18_CuIn8_rand1_opt
Cu9 In9 Zn16
Attempts: 2
ZnS16_CuIn9_rand1_opt
Cu10 In10 Zn14
Attempts: 1
ZnS14_CuIn10_rand1_opt
Cu11 In11 Zn12
Attempts: 1
ZnS12_CuIn11_rand1_opt
Cu12 In12 Zn10
Attempts: 4
ZnS10_CuIn12_rand1_opt
Cu13 In13 Zn8
Attempts: 1
ZnS8_CuIn13_rand1_opt
Cu14 In14 Zn6
Attempts: 4
ZnS6_CuIn14_rand1_opt
Cu15 In15 Zn4
Attempts: 1
ZnS4_CuIn15_rand1_opt
Cu16 In16 Zn2
Attempts: 2
ZnS2_CuIn16_rand1_opt


In [9]:
np.unique(test2, return_counts=True)

NameError: name 'test2' is not defined

In [44]:
H_num = 97
total_H_charge = 0
for i in range(67,0,-1):
    atom = test[i]
    if atom == 1:
        H_charge = 1.75
    elif atom == 2:
        H_charge = 1.5
    elif atom == 3:
        H_charge = 1.25
    
    if ((i+1) in corner_cations) or ((i+1) in surface_cations):
        print("{}\t{}".format(H_num, H_charge))
        H_num += 1
        total_H_charge += H_charge
    if ((i+1) in corner_cations):  # do a second time if on corner
        print("{}\t{}".format(H_num, H_charge))
        H_num += 1
        total_H_charge += H_charge

97	1.25
98	1.25
99	1.5
100	1.5
101	1.5
102	1.5
103	1.5
104	1.5
105	1.5
106	1.5
107	1.5
108	1.5
109	1.75
110	1.5
111	1.75
112	1.75
113	1.5
114	1.25
115	1.5
116	1.75
117	1.5
118	1.75
119	1.5
120	1.25
121	1.5
122	1.25
123	1.25
124	1.75
